## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-18-18-55-00 +0000,wsj,A federal court has blocked the use of a new T...,https://www.wsj.com/politics/policy/texas-new-...
1,2025-11-18-18-53-59 +0000,nyt,What’s the Most Elegant Way to Lie to My Co-Wo...,https://www.nytimes.com/2025/11/15/business/ly...
2,2025-11-18-18-52-13 +0000,nypost,Google’s Sundar Pichai warns of AI spending ‘i...,https://nypost.com/2025/11/18/business/google-...
3,2025-11-18-18-51-01 +0000,nyt,Live Updates: Trump Praises Saudi Prince and D...,https://www.nytimes.com/live/2025/11/18/world/...
4,2025-11-18-18-51-00 +0000,wsj,Inside the Israeli-Linked Flights Carrying Pal...,https://www.wsj.com/world/middle-east/inside-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2347/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,58
320,epstein,20
3,new,16
323,files,14
47,out,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
313,2025-11-17-21-29-00 +0000,wsj,Trump said he would sign legislation to releas...,https://www.wsj.com/politics/policy/trump-says...,138
97,2025-11-18-15-49-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,134
252,2025-11-18-02-00-00 +0000,wsj,President Trump’s grip on the GOP is showing s...,https://www.wsj.com/politics/policy/trumps-gri...,121
57,2025-11-18-17-18-09 +0000,nypost,NY Gov. Kathy Hochul enlists Wall Street execs...,https://nypost.com/2025/11/18/business/ny-gov-...,116
254,2025-11-18-01-48-49 +0000,nyt,Trump Has the Power to Release the Epstein Fil...,https://www.nytimes.com/2025/11/17/us/politics...,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
313,138,2025-11-17-21-29-00 +0000,wsj,Trump said he would sign legislation to releas...,https://www.wsj.com/politics/policy/trump-says...
57,58,2025-11-18-17-18-09 +0000,nypost,NY Gov. Kathy Hochul enlists Wall Street execs...,https://nypost.com/2025/11/18/business/ny-gov-...
144,51,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...
0,49,2025-11-18-18-55-00 +0000,wsj,A federal court has blocked the use of a new T...,https://www.wsj.com/politics/policy/texas-new-...
8,47,2025-11-18-18-46-44 +0000,nypost,Trump’s border czar Tom Homan warns NY will se...,https://nypost.com/2025/11/18/us-news/trumps-b...
214,41,2025-11-18-09-18-41 +0000,wapo,U.N. Security Council approves Trump’s 20-poin...,https://www.washingtonpost.com/national-securi...
309,37,2025-11-17-21-50-00 +0000,wsj,"Market Rout Intensifies, Sweeping Up Everythin...",https://www.wsj.com/finance/stocks/market-rout...
174,37,2025-11-18-11-37-32 +0000,nypost,James Comey trial judge stays order for feds t...,https://nypost.com/2025/11/18/us-news/james-co...
97,34,2025-11-18-15-49-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
17,33,2025-11-18-18-36-00 +0000,nypost,"Dow plunges 300 points on fears of AI bubble, ...",https://nypost.com/2025/11/18/business/dow-plu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
